In [1]:
import rospy
import pickle
import moveit_commander 
from sensor_msgs.msg import JointState
from moveit_msgs.msg import DisplayTrajectory, RobotTrajectory
import random


In [2]:
import moveit_msgs.msg
rospy.init_node('notebook')
state_pub = rospy.Publisher("notebook_state", moveit_msgs.msg.DisplayRobotState, queue_size=1, latch=True)
traj_pub = rospy.Publisher("notebook_traj", moveit_msgs.msg.DisplayTrajectory, queue_size=1, latch=True)

In [24]:
with open("/home/hri25-group3/ros/src/safety_perception/joint_snapshot_final.pkl", "rb") as f:
        data = pickle.load(f)

In [23]:
with open("/home/hri25-group3/ros/src/safety_perception/final_pairs_fast.pkl", "rb") as f:
        plans = pickle.load(f)

In [25]:
data[0]


{'t': 1762939658.5318162,
 'names': ['arm_shoulder_pan_joint',
  'arm_shoulder_lift_joint',
  'arm_elbow_joint',
  'arm_wrist_1_joint',
  'arm_wrist_2_joint',
  'arm_wrist_3_joint'],
 'q': [-3.484718624745504,
  -3.1300374470152796,
  -0.011849907226860523,
  -1.5491812986186524,
  1.5686696767807007,
  -0.0009916464435022476]}

In [26]:
def display_state(s):    
    ds = moveit_msgs.msg.DisplayRobotState()
    ds.state.joint_state.name = s['names']
    ds.state.joint_state.position = s['q']
    state_pub.publish(ds)

display_state(data[5])

In [9]:
def display_plan(p):
    dtraj = moveit_msgs.msg.DisplayTrajectory()
    rtraj = moveit_msgs.msg.RobotTrajectory()
    rtraj.joint_trajectory = p.joint_trajectory
    dtraj.trajectory.append(rtraj)
    traj_pub.publish(dtraj)

display_plan(plans[0,5])

In [52]:
move = moveit_commander.MoveGroupCommander('arm')
move.set_max_velocity_scaling_factor(0.5)

move.set_named_target('ready')
move.go()


for i in range (len(data)-1):
    #start
    js1= JointState()
    js1.name=data[i]['names']
    js1.position=data[i]['q']
    move.set_joint_value_target(js1)
    move.set_max_velocity_scaling_factor(0.5)
    move.go()
    j=i+1
    for j in range (len(data)-1):
        #end
        js2= JointState()
        js2.name=data[i]['names']
        js2.position=data[i]['q']
        move.set_joint_value_target(js2)
        move.set_max_velocity_scaling_factor(0.5)
        move.go()
        #go back to start
        move.set_joint_value_target(js1)
        move.set_max_velocity_scaling_factor(0.5)
        move.go()
        
            


[INFO] [1762944868.156653768]: Ready to take commands for planning group arm.


MoveItCommanderException: Error setting joint target. Is the target within bounds?

In [27]:
plans.keys()

dict_keys([(2, 3), (3, 2), (2, 4), (4, 2), (2, 5), (5, 2), (2, 6), (6, 2), (2, 7), (7, 2), (2, 8), (8, 2), (2, 9), (9, 2), (2, 10), (10, 2), (2, 11), (11, 2), (2, 12), (12, 2), (2, 13), (13, 2), (2, 14), (14, 2), (2, 15), (15, 2), (2, 16), (16, 2), (2, 17), (17, 2), (2, 18), (18, 2), (2, 19), (19, 2), (3, 4), (4, 3), (3, 5), (5, 3), (3, 6), (6, 3), (3, 7), (7, 3), (3, 8), (8, 3), (3, 9), (9, 3), (3, 10), (10, 3), (3, 11), (11, 3), (3, 12), (12, 3), (3, 13), (13, 3), (3, 14), (14, 3), (3, 15), (15, 3), (3, 16), (16, 3), (17, 3), (18, 3), (19, 3), (5, 4), (4, 6), (6, 4), (4, 7), (7, 4), (8, 4), (4, 9), (9, 4), (4, 10), (10, 4), (4, 11), (11, 4), (4, 12), (12, 4), (13, 4), (14, 4), (4, 15), (15, 4), (16, 4), (4, 17), (17, 4), (18, 4), (4, 19), (19, 4), (5, 6), (6, 5), (7, 5), (5, 8), (8, 5), (5, 9), (9, 5), (10, 5), (5, 11), (11, 5), (5, 12), (12, 5), (5, 13), (13, 5), (5, 14), (14, 5), (5, 15), (15, 5), (5, 16), (16, 5), (5, 17), (17, 5), (18, 5), (19, 5), (7, 6), (8, 6), (9, 6), (10, 6)

In [65]:
move = moveit_commander.MoveGroupCommander('arm')
move.set_max_velocity_scaling_factor(0.1)

start= data[2]

js1= JointState()
js1.name= start['names']
js1.position=start['q']



[INFO] [1762945194.156847809]: Ready to take commands for planning group arm.


In [66]:
move.set_joint_value_target(js1)# go to start 0 position

move.go()

True

In [50]:
k0 = [k for k in plans.keys() if isinstance(k, tuple) and len(k) >= 2 and k[0] == 2]
print(k0)




[(2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13), (2, 14), (2, 15), (2, 16), (2, 17), (2, 18), (2, 19)]


In [29]:
if not k0:
    raise ValueError("No keys starting with 0 found.")

key = random.choice(k0)          # e.g., (0, 5)
plan = plans[key]                # access the plan
print("Picked key:", key)

endpoint=key[1]
#print(endpoint)

plans[key]


Picked key: (2, 8)


joint_trajectory: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "base"
  joint_names: 
    - arm_shoulder_pan_joint
    - arm_shoulder_lift_joint
    - arm_elbow_joint
    - arm_wrist_1_joint
    - arm_wrist_2_joint
    - arm_wrist_3_joint
  points: 
    - 
      positions: [-3.100900475178854, -2.6826077900328578, 0.05143720308412725, -1.5458531242660065, 1.5680562257766724, -0.0009525457965295914]
      velocities: [0.009054526082562618, 0.016869357547019008, -0.06283185307179588, 0.012717876366346909, -0.03622909666989687, -3.629867698293182e-06]
      accelerations: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
      effort: []
      time_from_start: 
        secs: 0
        nsecs:         0
    - 
      positions: [-3.096373212137572, -2.674173111259348, 0.0200212765482293, -1.539494186082833, 1.5499416774417238, -0.000954360730378738]
      velocities: [0.09054526082562742, 0.16869357547018904, -0.6283185307179586, 0.12717876366346914, -0.3622909666989682

In [43]:
rtraj.joint_trajectory = plans[key].joint_trajectory


In [44]:
jt = rtraj.joint_trajectory
assert len(jt.points) > 0, "Chosen trajectory has no points."

# go to first waypoint so controllers don't abort
start_map = dict(zip(jt.joint_names, jt.points[0].positions))
move.set_joint_value_target(start_map)
ok = move.go(wait=True); move.stop()
print("Moved to start:", ok)

# execute the trajectory
ok = move.execute(rtraj, wait=True); move.stop()
print("Executed trajectory:", ok)


[INFO] [1762773693.710934252]: ABORTED: CONTROL_FAILED
Moved to start: False
[INFO] [1762773693.714344806]: ABORTED: CONTROL_FAILED
Executed trajectory: False


In [57]:
def has_outgoing(plans, node):
    # True if there exists any key (node, k) in plans
    return any(k[0] == node for k in plans)

In [59]:
def pick_edge_with_future(plans, i, avoid_edge=None):
    """Pick (i->j) preferring j that has outgoing edges. Avoid a specific edge if given."""
    outs = [k for k in plans if k[0] == i and k != avoid_edge]
    if not outs:
        return None
    # prefer edges whose destination isn't a dead-end
    good = [k for k in outs if has_outgoing(plans, k[1])]
    return random.choice(good if good else outs)[1]

In [60]:
cur_idx = 2

In [64]:
next_idx = pick_edge_with_future(plans, cur_idx)
display_state(data[next_idx])
# TODO: make this next line execution and check for success before going on
display_plan(plans[cur_idx,next_idx])


#traj_slow = move.retime_trajectory(display_state(cur_idx), tr,velocity_scaling_factor=0.5,acceleration_scaling_factor=0.5)
move.execute(plans[cur_idx,next_idx],wait=True)
cur_idx = next_idx

[INFO] [1762945171.028718046]: ABORTED: CONTROL_FAILED


In [67]:
r = rospy.Rate(0.7)
while not rospy.is_shutdown():
    next_idx = pick_edge_with_future(plans, cur_idx)
    display_state(data[next_idx])
    # TODO: make this next line execution and check for success before going on
    display_plan(plans[cur_idx,next_idx])
    move.execute(plans[cur_idx,next_idx],wait=True)
    cur_idx = next_idx
    r.sleep()

[INFO] [1762945211.354123707]: ABORTED: CONTROL_FAILED
[INFO] [1762945212.782409780]: ABORTED: CONTROL_FAILED


KeyboardInterrupt: 

In [86]:
# choose from current index (e.g., cur_idx = 0)
key = pick_edge_with_future(plans, cur_idx)
if key is None:
    raise ValueError(f"No outgoing trajectories from {cur_idx}")

rtraj = plans[key]  # RobotTrajectory

# your existing execution snippet:
jt = rtraj.joint_trajectory
if not jt.points:
    raise ValueError("Chosen trajectory has no points.")

start_map = dict(zip(jt.joint_names, jt.points[0].positions))
move.set_joint_value_target(start_map)
ok = move.go(wait=True); move.stop()
print("Moved to start:", ok)

ok = move.execute(rtraj, wait=True); move.stop()
print("Executed trajectory:", ok)

# update to the new endpoint
cur_idx = key[1]
print("Now at index:", cur_idx)


ValueError: No outgoing trajectories from 8